In [1]:
import pandas as pd
import numpy as np
import json
import os
from Agents import *

## Beautiful Soup + Selenium to get new weights from 17lands

In [3]:
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

chrome_options = Options()

chrome_options.add_argument("--headless")

archetypes = ['WU','WB','WR','WG','UB','UR','UG','BR','BG','RG']

expansion_string = 'VOW'


#store the url of the first page
url = 'https://www.17lands.com/card_ratings'
#define the executable path on the webdriver using the file's destination 
driver = webdriver.Chrome(executable_path='C:/Users/trist/Downloads/chromedriver.exe',options=chrome_options)
try:
    #get the url in the driver, use either URL or URL_two depending on which page you're trying to scrape
    driver.get(url)

    #Now let's make sure that we are choosing the right set
    select = Select(driver.find_element_by_id('expansion'))

    # select by visible text
    select.select_by_visible_text(expansion_string)

    #Wait for our table body to load
    table = WebDriverWait(driver, 20).until(EC.visibility_of_element_located(
        (By.XPATH, '//tbody')))

    #Grab the page source
    html=driver.page_source

    #5 second cooldown 
    time.sleep(5)

    #Create list of html tables that we will later iterate through
    html_list = []
    for arch in archetypes:
        #Now let's make sure that we are choosing the right set
        select = Select(driver.find_element_by_id('deck_color'))

        # select by visible text
        select.select_by_visible_text(arch)

        #Wait for our table body to load
        table = WebDriverWait(driver, 20).until(EC.visibility_of_element_located(
        (By.XPATH, '//tbody')))

        #Grab the page source
        new_html=driver.page_source

        html_list.append(new_html)

        #5 second cooldown 
        time.sleep(5)

    driver.quit()
except TimeoutError as tie:
    #Error Handling 
    print("Exception " +tie+ 'has occured')
    driver.quit()


In [7]:
div = soup.select_one('table')
t2 = pd.read_html(div.prettify())[0]

for idx, tbl in enumerate(html_list):
    soup = BeautifulSoup(tbl,'html.parser')
    div = soup.select_one('table')
    new_table = pd.read_html(div.prettify())[0].loc[:,['Name','# GP','GP WR']]
    #new_table['IWD'] = np.where((new_table['# Picked']>1700)&(new_table['Rarity']!='M'),new_table['IWD'],0)
    new_table['GP WR'] = np.where((new_table['# GP']>=800),new_table['GP WR'],0)
    #new_table['Seen_rate'] = new_table['# GP'] /new_table['# Seen'] 
    new_table = new_table.loc[:,['Name','GP WR']]
    t2 = t2.merge(right=new_table,how='inner',on='Name',suffixes=[None," " + archetypes[idx]])

In [16]:
# #This cell actually does things.... here we will impute null values using the card's avg IWD
impute_colnames = t2.iloc[:,-len(archetypes):].columns.values
t2copy = t2.copy()

# #Make all nulls np.nan so we can remove them with the min with a masked array
# t2copy['IWD'] = t2copy['IWD'].fillna('999.9pp')

# #Remove the pp suffix from our numbers and convert to float
# t2copy['IWD'] = t2copy['IWD'].str.replace('pp','').astype('float')

# #Create masked array that will look for all positions where we have np.nan and replace it with the min
# #The intuition here is if we don't have enough data for the IWD of a card at all, people are against picking it all, thus meaning it must have a low IWD
# masked_iwd = t2copy['IWD'].mask(
#     t2copy['IWD'].eq(999.9),
#     t2copy['IWD'].min())

# #Replace old IWD column with our masked_iwd, which is a float
# t2copy['IWD'] = masked_iwd


for c in impute_colnames:

    #Generally, we will fill the specialized cols with the overall IWD when the sample is small
    t2copy[c] = t2copy[c].fillna(0).astype('string')

    #For those we aren't replacing, we also need to clean off the 'pp' string and convert datatypes
    t2copy[c] = t2copy[c].str.replace('%','').astype('float')

In [19]:
dummies_df = pd.DataFrame(output, columns=tt.columns.values)

final_weights_df = pd.concat([tt, dummies_df]).sort_values(by='Name').reset_index(drop=True)

In [20]:
final_weights_df.shape[0]

272

In [21]:
final_weights_df = final_weights_df.loc[:, final_weights_df.columns != 'IWD']

In [22]:
final_weights_df

,GP WR WU,GP WR WB,GP WR WR,GP WR WG,GP WR UB,GP WR UR,GP WR UG,GP WR BR,GP WR BG,GP WR RG,Name,Color
0,0.0,0.0,56.4,0.0,0.0,58.0,0.0,57.9,0.0,56.5,Abrade,R
1,56.6,54.8,55.6,53.8,0.0,0.0,0.0,0.0,0.0,0.0,Adamant Will,W
2,0.0,54.4,0.0,0.0,51.0,0.0,0.0,55.0,50.4,0.0,Aim for the Head,B
3,0.0,0.0,0.0,0.0,0.0,56.1,0.0,0.0,0.0,0.0,Alchemist's Gambit,UR
4,56.7,0.0,0.0,0.0,53.7,57.1,52.7,0.0,0.0,0.0,Alchemist's Retrieval,U
...,...,...,...,...,...,...,...,...,...,...,...,...
267,0.0,0.0,0.0,53.2,0.0,0.0,54.3,0.0,52.5,55.7,Witch's Web,G
268,56.6,0.0,0.0,0.0,52.1,56.7,48.0,0.0,0.0,0.0,Witness the Future,U
269,0.0,0.0,0.0,55.3,0.0,0.0,54.8,0.0,54.0,56.0,Wolf Strike,G
270,0.0,0.0,0.0,53.8,0.0,0.0,52.6,0.0,51.4,56.0,Wolfkin Outcast,G


In [14]:
final_weights_df.to_csv('VOM_Weights_default__GPWR_df.csv', index=False)